# Moving Window Regressions and Map Visualization

Date: 04/15/2023

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime as dt
%matplotlib inline

# Machine Learning
import scipy as sp
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

# Map
from PIL import Image
from urllib.request import urlopen
import json
import plotly.express as px
import kaleido
import geopandas
from geopandas.tools import sjoin
import folium
import shapely
from shapely import wkt
from shapely.geometry import shape

In [ ]:
# Parameters
threshold_distance = 50
threshold_no_counties = 4
threshold_obs = 30

### 1. Read in NPP, GPP, Yield, and Weather Data

In [ ]:
def read_concat(file_name, crop, start_year, end_year):
    """
    Read in a set of data sets and concatenate them into one dataframe.
    """
    cwd = os.path.dirname(os.getcwd())
    filename = 'Data_original\\' + file_name + str(start_year) + '_' + crop + '.csv'
    location = os.path.join(cwd, filename)
    output_df = pd.read_csv(location, header=0)

    for y in range(start_year+1, end_year+1):
        filename = 'Data_original\\' + file_name + str(y) + '_' + crop + '.csv'
        location = os.path.join(cwd, filename)
        df = pd.read_csv(location, header=0)
        output_df =  pd.concat([output_df, df], ignore_index=True)
        
    return output_df

In [ ]:
# Read in yield data by county.
cwd = os.path.dirname(os.getcwd())
filename = 'Data_original\\corn_yield_data.csv'
location = os.path.join(cwd, filename)
yield_df = pd.read_csv(location, header=0)
yield_df = yield_df[['year', 'state_ansi', 'county_ansi', 'Value']].copy()

# Read in GPP data by county.
GPP_acc = read_concat('GPPintegral', 'Corn', 2001, 2020)
GPP_acc.drop(columns=['system:index', '.geo', 'QC_sum'], inplace=True)
GPP_acc.dropna(subset=['GPP_sum'], inplace=True)
GPP_acc.reset_index(drop=True, inplace=True)
corn_df = GPP_acc.merge(yield_df, 
                        left_on=['STATEFP', 'COUNTYFP', 'Year'], 
                        right_on=['state_ansi', 'county_ansi', 'year'], 
                        how='outer')

# Read in NPP data by county.
NPP_acc = read_concat('NPPintegral', 'Corn', 2001, 2019)
NPP_acc = NPP_acc[['STATEFP', 'COUNTYFP', 'Year', 'annualNPP_sum']]
NPP_acc.dropna(subset=['annualNPP_sum'], inplace=True)
NPP_acc.reset_index(drop=True, inplace=True)
corn_df = corn_df.merge(NPP_acc, 
                        on=['STATEFP', 'COUNTYFP', 'Year'], 
                        how='outer')

# Keep only useful variables
corn_df.Year.fillna(corn_df.year, inplace=True)
corn_df.COUNTYFP.fillna(corn_df.county_ansi, inplace=True)
corn_df.STATEFP.fillna(corn_df.state_ansi, inplace=True)
corn_df.drop(columns=['year', 'state_ansi', 'county_ansi'], inplace=True)
corn_df.columns = ['AFFGEOID', 'ALAND', 'AWATER', 'COUNTYFP', 'COUNTYNS', 'CropLand',
                   'GEOID', 'GPP_integral', 'LSAD', 'NAME', 'STATEFP', 'Year', 'Yield',
                   'NPP_integral']
corn_df.drop(columns=['AFFGEOID', 'ALAND', 'AWATER', 'COUNTYNS', 'LSAD'], 
             inplace=True)

# Read in weather data by county.
filename = 'Data_original\\corn_weather.csv'
location = os.path.join(cwd, filename)
weather_df_corn = pd.read_csv(location, header=0)
corn_df = corn_df.merge(weather_df_corn, 
                        left_on=['STATEFP', 'COUNTYFP', 'Year'], 
                        right_on=['statefp', 'countyfp', 'year'], 
                        how='left')

# Create useful variables and drop redundant variables
corn_df['fips'] = corn_df.COUNTYFP + corn_df.STATEFP*1000
corn_df['fips'] = ['{0:05}'.format(int(x)) 
                   for x in corn_df['fips']]
corn_df['CUE'] = corn_df['NPP_integral']/corn_df['GPP_integral']
corn_df.drop(columns=['statefp', 'countyfp', 'Unnamed: 0', 
                      'year', 'STATEFP', 'COUNTYFP',
                      'GEOID', 'NAME'], inplace=True)
corn_df = corn_df.loc[corn_df.Year>2000]

In [ ]:
# Merge corn_df with county geometries 
# and make it a geodataframe.
filename = 'Data_original\\cb_2018_us_county_20m.zip'
location = os.path.join(cwd, filename)
counties = geopandas.read_file(location)
corn_df = corn_df.merge(counties, 
                        left_on=['fips'], 
                        right_on=['GEOID'],
                        how='left')
corn_df = geopandas.GeoDataFrame(corn_df, 
                                 geometry=corn_df.geometry,
                                 crs=counties.crs)
counties.to_crs(crs=3857, inplace=True)
corn_df.to_crs(crs=3857, inplace=True)

# Drop redundant variables
corn_df.drop(columns=['CropLand', 'GPP_integral', 'NPP_integral', 
                      'fips', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 
                      'AFFGEOID','NAME', 'LSAD', 'ALAND', 'AWATER'],
             inplace=True)

### 2. Moving Window Regressions

In [ ]:
corn_coef = []

for i in range(len(counties)):
    d = {}
    df = corn_df.loc[corn_df.distance(counties.iloc[i].geometry)
                     <threshold_distance].copy()
    df_yield = df.loc[:, ['GEOID','Yield','edd','gdd','prcp']].dropna()
    df_CUE = df.loc[:, ['GEOID','CUE','edd','gdd','prcp']].dropna()
    d['fips'] = counties.iloc[i].GEOID
    
    # Data of county i is available;
    # more than threshold_no_counties counties are merged due to the distance criteria;
    # total number of observations is more than threshold_obs.
    if (sum(df_yield.GEOID==counties.iloc[i].GEOID)>0) and \
    (len(df_yield.groupby('GEOID'))>threshold_no_counties) and \
    (len(df_yield)>threshold_obs):
        mod = smf.ols(formula='Yield ~ edd + gdd + prcp', \
              data=df_yield, missing='drop')
        res = mod.fit(cov_type='HC3')
        d['yield_edd'] = res.params.edd
        d['yield_gdd'] = res.params.gdd
        d['yield_prcp'] = res.params.prcp
        d['yield_edd_p'] = res.pvalues.edd
        d['yield_gdd_p'] = res.pvalues.gdd
        d['yield_prcp_p'] = res.pvalues.prcp
        
    if (sum(df_CUE.GEOID==counties.iloc[i].GEOID)>0) and \
    (len(df_CUE.groupby('GEOID'))>threshold_no_counties) and \
    (len(df_CUE)>threshold_obs):
        mod = smf.ols(formula='CUE ~ edd + gdd + prcp', \
              data=df_CUE, missing='drop')
        res = mod.fit(cov_type='HC3')
        d['CUE_edd'] = res.params.edd
        d['CUE_gdd'] = res.params.gdd
        d['CUE_prcp'] = res.params.prcp
        d['CUE_edd_p'] = res.pvalues.edd
        d['CUE_gdd_p'] = res.pvalues.gdd
        d['CUE_prcp_p'] = res.pvalues.prcp
         
    corn_coef.append(d)
    
corn_coef = pd.DataFrame(corn_coef)

# For comparison of the responses of CUE and yield to weather conditions,
# drop those regression results missing one or both regressions.
corn_coef.dropna(subset=['yield_gdd', 'CUE_gdd'], inplace=True)

### 3. Result Visualization

In [ ]:
# Load in county base map.
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
corn_coef.columns

#### 3.1. Coefficient Map

In [ ]:
# Create a 
def posi_nega_zero(coef, pvalue):
    a = np.sign(coef)*(abs(pvalue)<0.1)
    if a == 0:
        return('0')
    elif a > 0:
        return('+')
    else:
        return('-')

In [ ]:
corn_coef['yield_edd_sign'] = [posi_nega_zero(*a) 
                               for a in tuple(
                                   zip(corn_coef['yield_edd'], 
                                       corn_coef['yield_edd_p']))]
corn_coef['yield_gdd_sign'] = [posi_nega_zero(*a) 
                               for a in tuple(
                                   zip(corn_coef['yield_gdd'], 
                                       corn_coef['yield_gdd_p']))]
corn_coef['yield_prcp_sign'] = [posi_nega_zero(*a) 
                                for a in tuple(
                                    zip(corn_coef['yield_prcp'], 
                                        corn_coef['yield_prcp_p']))]
corn_coef['CUE_edd_sign'] = [posi_nega_zero(*a) 
                             for a in tuple(
                                 zip(corn_coef['CUE_edd'], 
                                     corn_coef['CUE_edd_p']))]
corn_coef['CUE_gdd_sign'] = [posi_nega_zero(*a) 
                             for a in tuple(
                                 zip(corn_coef['CUE_gdd'], 
                                     corn_coef['CUE_gdd_p']))]
corn_coef['CUE_prcp_sign'] = [posi_nega_zero(*a) 
                              for a in tuple(
                                  zip(corn_coef['CUE_prcp'], 
                                      corn_coef['CUE_prcp_p']))]

In [ ]:
corn_coef['gdd_category'] = corn_coef['yield_gdd_sign'] + \
                            corn_coef['CUE_gdd_sign']
corn_coef['prcp_category'] = corn_coef['yield_prcp_sign'] + \
                            corn_coef['CUE_prcp_sign']
corn_coef['edd_category'] = corn_coef['yield_edd_sign'] + \
                            corn_coef['CUE_edd_sign']

In [ ]:
len(corn_coef[corn_coef['yield_gdd_sign']=='+'])/len(corn_coef)

In [ ]:
len(corn_coef[corn_coef['yield_prcp_sign']=='+'])/len(corn_coef)

In [ ]:
fig = px.choropleth(corn_coef, 
                    geojson=counties, 
                    locations='fips', 
                    color='gdd_category',
                    color_discrete_map={'--':'#660066', 
                                        '-0':'#f66d9b', 
                                        '0-':'#6574cd',
                                        '00':'rgb(224,224,224)',
                                        '+0':'#38c172',
                                        '0+':'#f6993f',
                                        '++':'#FFFF00',
                                        '+-':'blue',
                                        '-+':'red'
                                       },
                    scope="usa",
                    labels={'gdd_category':'yield/CUE'}
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(font=dict(size=20, color='black'))
fig.show()
filename = 'figures\\pnz_gdd.png'
location = os.path.join(cwd, filename)
fig.write_image(location)

In [ ]:
fig = px.choropleth(corn_coef, 
                    geojson=counties, 
                    locations='fips', 
                    color='prcp_category',
                    color_discrete_map={'--':'#660066', 
                                        '-0':'#f66d9b', 
                                        '0-':'#6574cd',
                                        '00':'rgb(224,224,224)',
                                        '+0':'#38c172',
                                        '0+':'#f6993f',
                                        '++':'#FFFF00',
                                        '+-':'blue',
                                        '-+':'red'
                                       },
                    scope="usa",
                    labels={'prcp_category':'yield/CUE'}
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(font=dict(size=20, color='black'))
fig.show()
filename = 'figures\\pnz_prcp.png'
location = os.path.join(cwd, filename)
fig.write_image(location)

In [ ]:
fig = px.choropleth(corn_coef, 
                    geojson=counties, 
                    locations='fips', 
                    color='edd_category',
                    color_discrete_map={'--':'#660066', 
                                        '-0':'#f66d9b', 
                                        '0-':'#6574cd',
                                        '00':'rgb(224,224,224)',
                                        '+0':'#38c172',
                                        '0+':'#f6993f',
                                        '++':'#FFFF00',
                                        '+-':'blue',
                                        '-+':'red'
                                       },
                    scope="usa",
                    labels={'edd_category':'yield/CUE'}
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(font=dict(size=20, color='black'))
fig.show()
filename = 'figures\\pnz_edd.png'
location = os.path.join(cwd, filename)
fig.write_image(location)

In [ ]:
filename = 'figures\\pnz_gdd.png'
location = os.path.join(cwd, filename)
img1 = Image.open(location)

filename = 'figures\\pnz_prcp.png'
location = os.path.join(cwd, filename)
img2 = Image.open(location)

fig, axs = plt.subplots(1, 2,
                        figsize=(24, 10))

axs[0].set_title('(a) Yield and CUE vs. GDD', fontsize = 25)
axs[0].imshow(img1)
axs[0].axis('off')

axs[1].set_title('(b) Yield and CUE vs. precipitation', fontsize = 25)
axs[1].imshow(img2)
axs[1].axis('off')

fig.tight_layout()
plt.show()

In [ ]:
filename = 'figures\\MovingWindowReg.png'
location = os.path.join(cwd, filename)
fig.savefig(location)